In [1]:
from teadata import DataEngine
from teadata.classes import haversine_miles, inspect_object
import pandas as pd

# Set option to display all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Enter the district name or district number of the school district you wish to analyze
DISTRICT_NAME = "Austin ISD"

# Instantiate DataEngine object, a layer over the District and Campus objects
repo = DataEngine.from_snapshot(search=True)

# Inspect how many objects are loaded
print(
    f"Loaded {len(repo.districts)} districts/charters and {len(repo.campuses)} campuses"
)

Loaded 1207 districts/charters and 9739 campuses


In [2]:
# Select a district using either the district name (case insensitive) or district number (any format works - integer and string with or without a leading apostrophe are both acceptable)
district = (repo >> ("district", DISTRICT_NAME)).first()
print("Example district:", district.name, district.district_number)

# Access canonical and enriched attributes (dot-syntax works if enrichment logic is wired correctly)
print(
    "\n2025 overall accountability rating:",
    district.rating,
    district.overall_rating_2025,
)

# Explore campuses inside the district
print(f"\n{district.name} has {len(district.campuses)} total campuses.")
print(f"\n{district.name} campus ratings:", district.campuses.value_counts("rating"))
print(f"\n{district.name} school types:", district.campuses.value_counts("school_type"))

Example district: Austin ISD '227901

2025 overall accountability rating: C C

Austin ISD has 127 total campuses.

Austin ISD campus ratings: [('C', 27), ('B', 25), ('F', 23), ('A', 22), ('D', 18), ('Not Rated', 7), ('Not Rated: Minimum Size rules', 1)]

Austin ISD school types: [('Elementary School', 81), ('High School', 17), ('Middle School', 17), ('Elementary/Secondary', 10), ('Junior High School', 2)]


In [3]:
# Select all of a district's campuses, then filter and sort and take the top 5 by enrollment, then find the nearest charter school to each campus with the same "school type" ("Elementary School", "Junior High School", "Middle School", "Elementary/Secondary", "High School")

# TODO: Is a better comparison to find overlapping grades served?

top5 = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda c: (c.enrollment or 0) > 0)
    >> ("sort", lambda c: c.enrollment or 0, True)
    >> ("take", 5)
    >> ("nearest_charter_same_type",)  # yields list of (campus, match, miles)
)

In [4]:
# Each data type includes .to_dict() (or .to_dicts() if a list) and .to_df() helper functions for exporting or working with query results
df = top5.to_df()  # auto-flattened & prefixed

# Define the columns we are interested in and the order we want
cols = [
    "campus_name",
    "campus_enrollment",
    "campus_rating",
    "campus_school_type",
    "match_name",
    "match_enrollment",
    "match_rating",
    "match_school_type",
    "distance_miles",
]

# Apply filter/sort to dataframe
df = df[cols]

# Display dataframe
df

,campus_name,campus_enrollment,campus_rating,campus_school_type,match_name,match_enrollment,match_rating,match_school_type,distance_miles
0,BOWIE H S,2876,A,High School,KIPP AUSTIN BRAVE,490,B,High School,6.143997
1,AKINS EARLY COLLEGE H S,2421,C,High School,KIPP AUSTIN BRAVE,490,B,High School,4.610669
2,AUSTIN H S,2323,B,High School,TEXANS CAN ACADEMY - AUSTIN,168,C,High School,3.464640
3,ANDERSON H S,2223,B,High School,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,116,A,High School,4.132389
4,MCCALLUM H S,1897,A,High School,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,116,A,High School,1.352387


In [5]:
# Get charter campuses physically inside that district’s boundary - either syntax works
charters_within = repo.charter_campuses_within(district)
charters_within = repo >> ("charters_within", district)

print(f"{district.name}: {len(charters_within)} charter campuses inside boundary")

Austin ISD: 38 charter campuses inside boundary


In [6]:
# Similar to before, export the result of our query to a dataframe for display after filtering and sorting the columns

df = charters_within.to_df()

cols = [
    "name",
    "campus_number",
    "district_number",
    "charter_type",
    "aea",
    "enrollment",
    "overall_rating_2025",
    "grade_range",
    "school_type",
    "school_status_date",
    "update_date",
]

df = df[cols]
df = df.sort_values(by="name")

df

,name,campus_number,district_number,charter_type,aea,enrollment,overall_rating_2025,grade_range,school_type,school_status_date,update_date
34,AUSTIN ACHIEVE FERGUSON,'227825006,'227825,OPEN ENROLLMENT CHARTER,N,73,Not Rated: Minimum Size rules,'EE-08,Elementary School,2023-08-09,2025-03-11
32,AUSTIN ACHIEVE H S,'227825004,'227825,OPEN ENROLLMENT CHARTER,N,581,B,'09-12,High School,2019-06-19,2025-03-11
30,AUSTIN ACHIEVE MIDDLE,'227825001,'227825,OPEN ENROLLMENT CHARTER,N,669,D,'05-08,Middle School,2012-08-27,2025-03-11
31,AUSTIN ACHIEVE NORTHEAST EL,'227825003,'227825,OPEN ENROLLMENT CHARTER,N,839,D,'PK-04,Elementary School,2018-08-06,2025-03-11
0,BASIS AUSTIN,'015834004,'015834,OPEN ENROLLMENT CHARTER,N,155,A,'03-12,Elementary/Secondary,2021-08-01,2025-03-11
1,BASIS AUSTIN PRI,'015834102,'015834,OPEN ENROLLMENT CHARTER,N,243,Not Rated: Minimum Size rules,'KG-02,Elementary School,2020-10-12,2025-03-11
19,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,'227817001,'227817,OPEN ENROLLMENT CHARTER,N,116,A,'09-12,High School,2016-08-03,2025-03-11
20,CEDARS INTERNATIONAL ACADEMY,'227817101,'227817,OPEN ENROLLMENT CHARTER,N,408,C,'PK-08,Elementary School,2000-08-18,2025-03-11
37,DIBOLL,'227827007,'227827,OPEN ENROLLMENT CHARTER,Y,75,Not Rated: Minimum Size rules,'09-12,High School,2021-11-15,2025-03-11
21,HARMONY SCHOOL OF EXCELLENCE - AUSTIN,'227816101,'227816,OPEN ENROLLMENT CHARTER,N,497,B,'06-12,Elementary/Secondary,2009-06-11,2025-03-11


In [7]:
rows = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda x: (x.enrollment or 0) > 0)
    >> ("sort", lambda x: x.enrollment or 0, True)
    >> ("take", 5)
    >> ("transfers_out",)
    >> ("where_to", lambda to: bool(getattr(to, "is_charter", True)))
)

cols = [
    "campus_campus_number",
    "campus_name",
    "campus_enrollment",
    "campus_overall_rating_2025",
    "campus_aea",
    "campus_grade_range",
    "campus_school_type",
    "campus_school_status_date",
    "campus_update_date",
    "to_campus_number",
    "to_name",
    "to_enrollment",
    "to_overall_rating_2025",
    "to_aea",
    "to_grade_range",
    "to_school_type",
    "to_charter_type",
    "to_is_charter",
    "to_school_status_date",
    "to_update_date",
    "count",
    "masked",
]

df = rows.to_df()

df = df[cols]

df

,campus_campus_number,campus_name,campus_enrollment,campus_overall_rating_2025,campus_aea,campus_grade_range,campus_school_type,campus_school_status_date,campus_update_date,to_campus_number,to_name,to_enrollment,to_overall_rating_2025,to_aea,to_grade_range,to_school_type,to_charter_type,to_is_charter,to_school_status_date,to_update_date,count,masked
0,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'227829001,VALOR SOUTH AUSTIN,1364,B,N,'KG-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2018-08-01,2025-03-11,42.0,False
1,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'072801113,PREMIER HIGH SCHOOL-AUSTIN SOUTH CAMPUS,143,A,Y,'06-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2011-05-31,2025-03-11,34.0,False
2,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'227803001,WAYSIDE SCI-TECH MIDDLE AND H S,554,C,N,'06-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2013-07-02,2025-03-11,12.0,False
3,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'227827001,THE EXCEL CENTER (FOR ADULTS),571,Not Rated: Minimum Size rules,Y,'09-AE,High School,OPEN ENROLLMENT CHARTER,True,2014-08-15,2025-03-11,10.0,False
4,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'227825004,AUSTIN ACHIEVE H S,581,B,N,'09-12,High School,OPEN ENROLLMENT CHARTER,True,2019-06-19,2025-03-11,NaN,True
5,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'227814001,CHAPARRAL STAR ACADEMY,364,A,N,'KG-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,1998-07-01,2025-03-11,NaN,True
6,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'108807037,IDEA BLUFF SPRINGS COLLEGE PREPARATORY,689,B,N,'06-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2016-08-09,2025-03-11,NaN,True
7,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'108807035,IDEA MONTOPOLIS COLLEGE PREPARATORY,835,B,N,'06-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2013-08-06,2025-03-11,NaN,True
8,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'108807090,IDEA PARMER PARK COLLEGE PREPARATORY,624,B,N,'06-11,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2019-07-31,2025-03-11,NaN,True
9,'227901013,BOWIE H S,2876,A,N,'09-12,High School,1987-12-18,2025-03-11,'221801022,ISCHOOL VIRTUAL ACADEMY OF TEXAS,778,D,N,'KG-12,Elementary/Secondary,OPEN ENROLLMENT CHARTER,True,2006-07-24,2025-03-11,NaN,True


In [8]:
for row in rows:
    print(row[0].overall_rating_2025)
    print(row[0].instruction_af_perc)
    

A


AttributeError: Campus has no attribute 'instruction_af_perc'